<a href="https://colab.research.google.com/github/orcacaor/aiffel_quest/blob/main/20251226_comp_clf_rubric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 제출용

In [ ]:
# 요구사항

# - 데이터 이해 및 탐색: Class의 균형과 데이터의 분포와 특성을 시각화를 통해서 파악했는가?
# - 모델 선택 및 비교: 3개 이상의 모델을 학습시키고 성능을 비교했는가?
# - 레이블 불균형을 처리하기 위한 기법을 시도했는가?
# - 여러 모델을 앙상블을 통해 결합하여 성능을 비교했는가?
# - Ridge/Lasso 등의 규제 추가, 트리 기반 모델의 깊이/학습률 등 하이퍼파라미터를 조정하여 성능 향상을 시도했는가?

In [ ]:
#testestesteststest